# Module 02: Zener Diodes and Engineered Breakdown

**Electricity & Semiconductors Course**

---

In the previous notebooks we treated reverse breakdown as something to avoid — exceeding a diode's reverse voltage limit destroys it. But what if we *designed* a diode to break down at a precise, repeatable voltage? That is exactly what a **Zener diode** is.

By the end of this notebook you will:
- Understand avalanche vs. Zener breakdown mechanisms
- Know how Zener diodes work as voltage regulators
- Design a simple Zener voltage regulator circuit with calculations
- Build and test a 5.1V regulator using a 1N4733A Zener
- Understand the limitations of Zener regulation

## Concept — Reverse Breakdown: Two Mechanisms

When a diode is reverse-biased beyond its limits, one of two breakdown mechanisms occurs:

### Avalanche Breakdown (> ~6V)
- Occurs in **lightly doped** junctions with **wide depletion regions**
- High reverse voltage accelerates minority carriers to enormous speeds
- Fast-moving carriers collide with lattice atoms and knock more electrons free
- Those new electrons are also accelerated, creating an **avalanche** of carriers
- Current multiplies exponentially — like a chain reaction
- Breakdown voltage has a **positive temperature coefficient** (increases with temperature)

### Zener Breakdown (< ~5V)
- Occurs in **heavily doped** junctions with **narrow depletion regions**
- The electric field across the thin depletion region becomes enormous
- The field is strong enough to **directly rip electrons from covalent bonds** (quantum tunneling)
- This is called **Zener tunneling** or **band-to-band tunneling**
- Breakdown voltage has a **negative temperature coefficient** (decreases with temperature)

### The Crossover
- Zener diodes rated at **~5V** have nearly **zero temperature coefficient** (both mechanisms contribute equally)
- This makes 5.1V Zeners particularly useful as voltage references
- In practice, all diodes that operate in reverse breakdown are called "Zener diodes" regardless of the actual mechanism

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Zener diode IV curve — including reverse breakdown region

k = 1.381e-23
q_e = 1.602e-19
T = 300
Vt = k * T / q_e

# Forward region
Vf = np.linspace(0, 0.85, 500)
Is = 1e-12
n = 1.5
If = Is * (np.exp(np.clip(Vf / (n * Vt), 0, 100)) - 1)

# Reverse region (before breakdown)
Vr_pre = np.linspace(-5.0, 0, 300)
Ir_pre = -Is * np.ones_like(Vr_pre)  # essentially flat at -Is

# Reverse breakdown region (Zener)
Vz = 5.1  # Zener voltage
Rz = 7.0  # Zener impedance (ohms)
Vr_break = np.linspace(-5.1, -5.0, 50)
Ir_break = -Is * np.ones_like(Vr_break)

# Past breakdown — current increases rapidly
Vr_zener = np.linspace(-8, -5.1, 300)
# Model: I = -(Vz_applied - Vz) / Rz
Ir_zener = (Vr_zener + Vz) / Rz  # This gives negative current values

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# --- Full IV curve ---
# Forward
ax1.plot(Vf * 1000, If * 1000, 'b-', lw=2.5)
# Reverse (pre-breakdown)
ax1.plot(Vr_pre * 1000, Ir_pre * 1000, 'b-', lw=2.5)
# Breakdown transition
ax1.plot(Vr_break * 1000, Ir_break * 1000, 'b-', lw=2.5)
# Zener region
ax1.plot(Vr_zener * 1000, Ir_zener * 1000, 'r-', lw=3)

ax1.axhline(y=0, color='gray', lw=0.5)
ax1.axvline(x=0, color='gray', lw=0.5)

ax1.set_xlabel('Voltage (mV)', fontsize=12)
ax1.set_ylabel('Current (mA)', fontsize=12)
ax1.set_title('Zener Diode IV Curve (1N4733A — 5.1V)', fontsize=14, fontweight='bold')
ax1.set_xlim(-8000, 1500)
ax1.set_ylim(-200, 100)
ax1.grid(True, alpha=0.3)

# Annotations
ax1.annotate('Forward region\n(normal diode)', xy=(700, 50), fontsize=10,
             color='blue', fontweight='bold',
             bbox=dict(boxstyle='round', fc='#d6eaf8'))
ax1.annotate('Zener breakdown\nregion (V_Z = 5.1V)', xy=(-7000, -150), fontsize=10,
             color='red', fontweight='bold',
             bbox=dict(boxstyle='round', fc='#fadbd8'))
ax1.annotate('', xy=(-5100, -10), xytext=(-5100, -60),
             arrowprops=dict(arrowstyle='->', color='red', lw=2))
ax1.text(-5100, -70, 'V_Z = 5.1V', fontsize=10, ha='center', color='red', fontweight='bold')

# --- Zoomed view of Zener region ---
Vz_detail = np.linspace(-6.5, -4.5, 500)
Iz_detail = np.where(Vz_detail < -Vz,
                      (Vz_detail + Vz) / Rz,
                      -Is)

ax2.plot(-Vz_detail * 1000, -Iz_detail * 1000, 'r-', lw=2.5)
ax2.set_xlabel('Reverse Voltage (mV)', fontsize=12)
ax2.set_ylabel('Reverse Current (mA)', fontsize=12)
ax2.set_title('Zoomed: Zener Breakdown Region', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)

# Show Zener impedance slope
ax2.annotate(f'Slope = 1/Rz\nRz = {Rz}Ω', xy=(5800, 80), fontsize=11,
             fontweight='bold', color='darkred',
             bbox=dict(boxstyle='round', fc='lightyellow'))

# Mark key points
ax2.axvline(x=5100, color='orange', linestyle='--', lw=1.5)
ax2.text(5150, 20, 'V_Z = 5.1V', fontsize=10, color='orange')

plt.tight_layout()
plt.show()

## The Material Science Why — Zener vs. Avalanche

The breakdown mechanism depends on the **doping concentration**, which determines the **depletion region width**:

| Property | Zener Mechanism | Avalanche Mechanism |
|----------|----------------|--------------------|
| Doping level | Heavy | Light |
| Depletion width | Narrow (<100nm) | Wide |
| Breakdown voltage | Low (< ~5V) | High (> ~6V) |
| Physical mechanism | Quantum tunneling | Impact ionization cascade |
| Temperature coefficient | Negative (~-2 mV/°C) | Positive (~+2 mV/°C) |

**Why heavy doping leads to tunneling:**

Heavy doping means many impurity atoms are packed closely. This creates a very thin depletion region. The electric field (E = V/d) becomes enormous even at modest voltages. When the field exceeds ~10⁶ V/cm, electrons can quantum tunnel directly through the bandgap barrier from the valence band to the conduction band.

**Why light doping leads to avalanche:**

Light doping creates a wide depletion region. The field is spread out over a larger distance, so it never gets strong enough for tunneling. But minority carriers crossing the depletion region gain enough kinetic energy from the field to knock electrons out of bonds when they collide with lattice atoms. Each collision creates more carriers, causing a multiplication avalanche.

**The 5V sweet spot:**

At approximately 5V, both mechanisms contribute equally. Their temperature coefficients cancel out, giving a nearly **zero temperature coefficient**. This is why 5.1V Zener diodes (like the 1N4733A) are popular as voltage references.

## Concept — Zener Voltage Regulator

The simplest voltage regulator uses a Zener diode and a series resistor:

```
         Rs
  Vin ──[████]──┬── Vout = Vz
                │
               ─┴─ Zener (1N4733A)
                ▲   Vz = 5.1V
                │
  GND ──────────┘
```

**How it works:**

1. The Zener operates in reverse breakdown — it **clamps** the output to V_Z
2. The series resistor Rs limits current through the Zener
3. If V_in increases, more current flows through the Zener (but V_out stays at V_Z)
4. If V_in decreases, less current flows through the Zener (V_out still stays at V_Z, down to a minimum)

**Design equations:**

- Current through Rs: `I_Rs = (V_in - V_Z) / Rs`
- This current splits: `I_Rs = I_Z + I_Load`
- For regulation, the Zener must stay in breakdown: `I_Z > I_Z(min)` (typically ~5mA)
- Power in Zener: `P_Z = V_Z × I_Z`
- Power in Rs: `P_Rs = (V_in - V_Z)² / Rs`

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def zener_regulator(Vin, Vz, Rs, Rload, Rz=7.0, Iz_min=1e-3):
    """
    Calculate Zener regulator output.
    
    Args:
        Vin: Input voltage (V)
        Vz: Zener voltage (V)
        Rs: Series resistance (Ohms)
        Rload: Load resistance (Ohms), use np.inf for no load
        Rz: Zener dynamic impedance (Ohms)
        Iz_min: Minimum Zener current for regulation (A)
    
    Returns:
        dict with Vout, Iz, Iload, Irs, Pz, Prs, regulated (bool)
    """
    # Total current from source
    # Vin = I_Rs * Rs + Vout
    # Vout = Vz + Iz * Rz (Zener model with impedance)
    # I_Rs = Iz + Iload = Iz + Vout/Rload
    
    # Simplified: assume Vout ≈ Vz when in regulation
    Irs = (Vin - Vz) / Rs
    Iload = Vz / Rload if Rload != np.inf else 0.0
    Iz = Irs - Iload
    
    regulated = Iz >= Iz_min
    
    if regulated:
        Vout = Vz + Iz * Rz  # slight variation due to Zener impedance
        # Recalculate more accurately
        Vout = Vz  # simplified
    else:
        # Zener drops out of regulation — acts like open circuit
        Vout = Vin * Rload / (Rs + Rload)
        Iz = 0
        Iload = Vout / Rload
        Irs = Iload
    
    Pz = Vz * Iz if regulated else 0
    Prs = Irs**2 * Rs
    
    return {
        'Vout': Vout, 'Iz': Iz, 'Iload': Iload, 'Irs': Irs,
        'Pz': Pz, 'Prs': Prs, 'regulated': regulated
    }

# --- Design calculation ---
Vin = 12.0     # Input voltage
Vz = 5.1       # Zener voltage
Iload = 20e-3  # Load current (20mA)
Iz_nom = 25e-3  # Desired Zener current (for good regulation)

# Calculate Rs
Irs_total = Iload + Iz_nom
Rs_calc = (Vin - Vz) / Irs_total

print("=" * 60)
print("ZENER REGULATOR DESIGN")
print("=" * 60)
print(f"Input voltage:    Vin = {Vin} V")
print(f"Zener voltage:    Vz  = {Vz} V")
print(f"Load current:     IL  = {Iload*1000:.0f} mA")
print(f"Desired Iz:       Iz  = {Iz_nom*1000:.0f} mA")
print(f"")
print(f"Total current:    Irs = IL + Iz = {Irs_total*1000:.0f} mA")
print(f"Series resistor:  Rs  = (Vin - Vz) / Irs")
print(f"                  Rs  = ({Vin} - {Vz}) / {Irs_total*1000:.0f}mA")
print(f"                  Rs  = {Rs_calc:.1f} Ω")
print(f"")
print(f"Use standard value: Rs = 150 Ω")
print(f"")

Rs = 150  # Standard value

# Verify design
result = zener_regulator(Vin, Vz, Rs, Vz / Iload)
print(f"Verification with Rs = {Rs}Ω, Rload = {Vz/Iload:.0f}Ω:")
print(f"  Vout = {result['Vout']:.2f} V")
print(f"  Iz   = {result['Iz']*1000:.1f} mA")
print(f"  IL   = {result['Iload']*1000:.1f} mA")
print(f"  P_Z  = {result['Pz']*1000:.0f} mW (max for 1N4733A: 500mW)")
print(f"  P_Rs = {result['Prs']*1000:.0f} mW")
print(f"  Regulated: {result['regulated']}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Plot output voltage vs load current — showing regulation limits

Vin = 12.0
Vz = 5.1
Rs = 150.0
Rz = 7.0   # Zener impedance

# Load current range: 0 to 60 mA
Il = np.linspace(0, 60e-3, 500)

# Total current through Rs
Irs = (Vin - Vz) / Rs  # Approximately constant when regulating

# Zener current
Iz = Irs - Il

# Output voltage:
# When Iz > 0: Vout ≈ Vz (with slight variation from Rz)
# When Iz ≈ 0: Zener drops out, circuit becomes voltage divider
Vout = np.where(Iz > 0,
                Vz + Iz * Rz * 0,  # Simplified: just Vz when regulating
                Vin - Il * Rs)       # Drops out of regulation

# More accurate: Vout = Vz + (change in Iz from nominal) * Rz
Iz_nominal = Irs  # Iz at no load
Vout_accurate = np.where(Iz > 1e-3,
                          Vz + (Iz - Iz_nominal/2) * Rz * 0.1,  # small variation
                          Vin - Il * Rs)  # out of regulation

# Find dropout point
Il_max = Irs - 1e-3  # max load current (leaving 1mA for Zener)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Vout vs Iload
ax1.plot(Il * 1000, Vout, 'b-', lw=2.5, label='V_out')
ax1.axhline(y=Vz, color='green', linestyle='--', lw=1, label=f'V_Z = {Vz}V')
ax1.axvline(x=Il_max * 1000, color='red', linestyle='--', lw=1.5, label=f'Max I_load = {Il_max*1000:.0f}mA')
ax1.fill_betweenx([0, 6], 0, Il_max * 1000, alpha=0.1, color='green', label='Regulated region')
ax1.fill_betweenx([0, 6], Il_max * 1000, 60, alpha=0.1, color='red', label='Out of regulation')

ax1.set_xlabel('Load Current (mA)', fontsize=12)
ax1.set_ylabel('Output Voltage (V)', fontsize=12)
ax1.set_title(f'Output Voltage vs Load Current\n(Vin={Vin}V, Rs={Rs}Ω)', fontsize=13, fontweight='bold')
ax1.set_xlim(0, 60)
ax1.set_ylim(0, 6)
ax1.legend(fontsize=10, loc='lower left')
ax1.grid(True, alpha=0.3)

# Power dissipation
Pz = np.where(Iz > 0, Vz * Iz, 0)
Prs = (Vin - Vout)**2 / Rs
Pload = Vout * Il

ax2.plot(Il * 1000, Pz * 1000, 'r-', lw=2, label='P_Zener')
ax2.plot(Il * 1000, Prs * 1000, 'b-', lw=2, label='P_Resistor')
ax2.plot(Il * 1000, Pload * 1000, 'g-', lw=2, label='P_Load')
ax2.axhline(y=500, color='red', linestyle=':', lw=1.5, label='Zener max (500mW)')

ax2.set_xlabel('Load Current (mA)', fontsize=12)
ax2.set_ylabel('Power (mW)', fontsize=12)
ax2.set_title('Power Dissipation vs Load Current', fontsize=13, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)
ax2.set_xlim(0, 60)

plt.tight_layout()
plt.show()

print(f"Maximum load current while maintaining regulation: {Il_max*1000:.0f} mA")
print(f"At no load: Zener dissipates {Vz * Irs * 1000:.0f} mW")
print(f"At max load: Zener dissipates {Vz * 1e-3 * 1000:.0f} mW")
print(f"\nKey insight: Worst-case Zener power is at NO LOAD (all current goes through Zener)")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
try:
    import ipywidgets as widgets
    from IPython.display import display
    HAS_WIDGETS = True
except ImportError:
    HAS_WIDGETS = False

def plot_zener_regulator(Vin=12.0, Rs=150, Rload=500):
    """Interactive Zener regulator visualization."""
    Vz = 5.1
    Rz = 7.0
    
    Irs = (Vin - Vz) / Rs
    Iload = Vz / Rload
    Iz = Irs - Iload
    
    regulated = Iz > 1e-3
    
    if regulated:
        Vout = Vz
    else:
        Vout = Vin * Rload / (Rs + Rload)
        Iz = 0
        Iload = Vout / Rload
        Irs = Iload
    
    Pz = Vz * Iz
    Prs = Irs**2 * Rs
    Pload = Vout * Iload
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 5))
    
    # Circuit status diagram
    ax1.set_xlim(0, 10)
    ax1.set_ylim(0, 8)
    ax1.axis('off')
    status_color = '#27ae60' if regulated else '#e74c3c'
    status_text = 'REGULATED' if regulated else 'NOT REGULATED'
    ax1.text(5, 7.5, f'Status: {status_text}', fontsize=16, fontweight='bold',
             ha='center', color=status_color)
    
    info_lines = [
        f'Vin = {Vin:.1f} V',
        f'Vout = {Vout:.2f} V',
        f'Rs = {Rs} Ω',
        f'Rload = {Rload} Ω',
        '',
        f'I_Rs = {Irs*1000:.1f} mA',
        f'I_Zener = {Iz*1000:.1f} mA',
        f'I_Load = {Iload*1000:.1f} mA',
        '',
        f'P_Zener = {Pz*1000:.0f} mW {"⚠ OVER 500mW!" if Pz > 0.5 else ""}',
        f'P_Rs = {Prs*1000:.0f} mW',
        f'P_Load = {Pload*1000:.0f} mW',
    ]
    for i, line in enumerate(info_lines):
        color = 'red' if 'OVER' in line else 'black'
        ax1.text(1, 6.5 - i * 0.55, line, fontsize=12, fontfamily='monospace', color=color)
    
    # Power pie chart
    if Pz + Prs + Pload > 0:
        powers = [Pz * 1000, Prs * 1000, Pload * 1000]
        labels = [f'Zener\n{Pz*1000:.0f}mW', f'Resistor\n{Prs*1000:.0f}mW',
                  f'Load\n{Pload*1000:.0f}mW']
        colors_pie = ['#e74c3c', '#3498db', '#27ae60']
        ax2.pie(powers, labels=labels, colors=colors_pie, autopct='%1.0f%%',
                startangle=90, textprops={'fontsize': 11})
        ax2.set_title('Power Distribution', fontsize=13, fontweight='bold')
    
    plt.tight_layout()
    plt.show()

if HAS_WIDGETS:
    vin_slider = widgets.FloatSlider(value=12.0, min=6.0, max=25.0, step=0.5,
                                      description='Vin (V):', style={'description_width': 'initial'})
    rs_slider = widgets.IntSlider(value=150, min=47, max=1000, step=1,
                                   description='Rs (Ω):', style={'description_width': 'initial'})
    rload_slider = widgets.IntSlider(value=500, min=50, max=5000, step=10,
                                      description='Rload (Ω):', style={'description_width': 'initial'})
    
    out = widgets.interactive_output(plot_zener_regulator,
                                      {'Vin': vin_slider, 'Rs': rs_slider, 'Rload': rload_slider})
    display(widgets.VBox([widgets.HBox([vin_slider, rs_slider, rload_slider]), out]))
else:
    print("ipywidgets not available — showing static example.")
    plot_zener_regulator(Vin=12.0, Rs=150, Rload=500)

## Experiment 1 — Measuring Zener Breakdown Voltage

**Equipment:** Bench power supply, 1N4733A Zener diode, 470Ω resistor, Klein MM300, breadboard

### Circuit

```
    Bench PSU
    ┌─────────┐
    │ 0-15V   │        470Ω          1N4733A
    │         │     ┌──[████]──┬────┤◄├────┐
    │        +├─────┤          │   (reverse │
    │         │     │         V_Z   biased) │
    │        -├─────┴──────────┴────────────┘
    └─────────┘
    
    NOTE: The Zener is REVERSE biased!
    Cathode (band) connects to the POSITIVE side.
    
    Measure V_Z across the Zener with multimeter.
```

### Steps

1. Build the circuit with the Zener **reverse biased** (cathode to positive)
2. Set bench supply to 0V, current limit to 50mA
3. Slowly increase voltage from 0V to 10V
4. At each voltage, measure voltage across the Zener diode
5. Note the point where the Zener voltage stops increasing (it should clamp at ~5.1V)

### Expected Results

| V_supply | V_Zener | I (calculated) | Notes |
|----------|---------|---------------|-------|
| 2V | ~2.0V | ~0 mA | Below breakdown |
| 4V | ~4.0V | ~0 mA | Below breakdown |
| 6V | ~5.1V | ~1.9 mA | In breakdown! |
| 8V | ~5.1V | ~6.2 mA | Regulating |
| 10V | ~5.1V | ~10.4 mA | Regulating |
| 12V | ~5.1V | ~14.7 mA | Regulating |

## Experiment 2 — 5.1V Regulator from 12V Supply

**Equipment:** Bench power supply, 1N4733A, 150Ω resistor (1/4W minimum), 470Ω load resistor, Klein MM300

### Circuit

```
    Bench PSU                                      Output
    ┌─────────┐      150Ω                          ┌───┐
    │  12.0V  │   ┌──[████]──┬────────────────────┤   │
    │         │   │          │                     │   │
    │        +├───┤         ─┴─ 1N4733A    [470Ω] │   │ V_out
    │         │   │          ▲ (reverse     LOAD  │   │
    │         │   │          │  biased)            │   │
    │        -├───┴──────────┴─────────────────────┤   │
    └─────────┘                                    └───┘
    
    Expected V_out ≈ 5.1V
    I_load = 5.1V / 470Ω ≈ 10.9 mA
    I_total = (12 - 5.1) / 150 ≈ 46 mA
    I_zener = 46 - 10.9 ≈ 35 mA
    P_zener = 5.1 × 35mA ≈ 179 mW (within 500mW limit)
```

### Steps

1. Build the circuit. Double-check Zener is **reverse biased**
2. Set bench supply to 12.0V, current limit 100mA
3. **Measure V_out** — should be close to 5.1V
4. **Measure current** from supply (read from bench supply display)

## Experiment 3 — Line Regulation (Varying Input Voltage)

Using the same circuit from Experiment 2 (with 470Ω load):

1. Vary the input voltage from 7V to 20V in 1V steps
2. At each step, measure V_out

### Expected Results

| V_in | V_out (expected) | V_out (measured) | Notes |
|------|-----------------|-----------------|-------|
| 7V | ~5.1V | _______ | Barely regulating |
| 8V | ~5.1V | _______ | |
| 10V | ~5.1V | _______ | |
| 12V | ~5.1V | _______ | Nominal |
| 15V | ~5.1V | _______ | |
| 18V | ~5.1V | _______ | |
| 20V | ~5.1V | _______ | Check Pz! |

**Watch for:** The output should stay very close to 5.1V across the entire range. Small variations (tens of millivolts) are due to the Zener's dynamic impedance.

## Experiment 4 — Load Regulation (Varying Load)

With V_in fixed at 12V, change the load resistor:

| R_load | I_load (calc) | V_out (expected) | V_out (measured) | Notes |
|--------|-------------|-----------------|-----------------|-------|
| No load (open) | 0 mA | ~5.1V | _______ | All current through Zener |
| 1kΩ | 5.1 mA | ~5.1V | _______ | |
| 470Ω | 10.9 mA | ~5.1V | _______ | |
| 220Ω | 23.2 mA | ~5.1V | _______ | |
| 150Ω | 34.0 mA | ~5.1V | _______ | Near Iz = 0, dropping out! |
| 100Ω | 51 mA | < 5.1V! | _______ | Iz < 0 means Zener is OFF |

**Key observation:** When the load demands more current than Rs can supply while keeping the Zener in breakdown, the regulator **drops out**. The output voltage drops below 5.1V.

## Concept — Limitations of Zener Regulation

Zener regulators are simple but have serious limitations:

1. **Efficiency is poor:** The series resistor and Zener constantly dissipate power, even at no load. At no load, ALL power is wasted.

2. **Load regulation is limited:** The maximum load current is constrained by Rs and the minimum Zener current.

3. **Line regulation is imperfect:** Zener impedance (Rz) means Vout varies slightly with current changes.

4. **Power waste increases with voltage difference:** Higher Vin means more power burned in Rs.

5. **Not suitable for high-current loads:** At 1A load, the series resistor would dissipate enormous power.

**This is why real voltage regulators exist** — ICs like the 7805 (linear regulator) or buck converters (switching regulators) solve these problems. But they all build on the same fundamental concepts we learned here.

| Regulator Type | Efficiency | Complexity | Noise |
|---------------|-----------|-----------|-------|
| Zener + resistor | 10-30% | Very low | Low |
| Linear regulator (7805) | 30-60% | Low | Very low |
| Switching regulator | 80-95% | Medium | Higher |

## Simulation

**Zener Voltage Regulator:**  
[https://www.falstad.com/circuit/circuitjs.html?ctz=CQAgjCAMB0l3BWcMBMcUHYMGZIA4UA2ATmIxAUgoqoQFMBaMMAKACURcQAWEPbPHigpyUJIOHjJKFDBmTZ-AFoBmAQwA2AZz6rRG6VJlygA](https://www.falstad.com/circuit/circuitjs.html?ctz=CQAgjCAMB0l3BWcMBMcUHYMGZIA4UA2ATmIxAUgoqoQFMBaMMAKACURcQAWEPbPHigpyUJIOHjJKFDBmTZ-AFoBmAQwA2AZz6rRG6VJlygA)

Try:
- Change the input voltage and watch V_out stay constant
- Decrease the load resistance until the Zener drops out
- Watch the current split between Zener and load

## Datasheet Connection — 1N4733A Key Parameters

| Parameter | Symbol | Value | What It Means |
|-----------|--------|-------|---------------|
| Zener voltage | V_Z | 5.1V (±5%) | Breakdown voltage at test current |
| Test current | I_ZT | 49 mA | Current at which V_Z is specified |
| Max Zener current | I_ZM | 91 mA | Maximum continuous reverse current |
| Zener impedance | Z_ZT | 7.0Ω | Dynamic impedance at test current |
| Max power dissipation | P_D | 1W | Max power (derate above 50°C) |
| Temp coefficient | T_C | +0.058 %/°C | Near-zero for 5.1V (sweet spot) |
| Leakage current | I_R | 5 μA @ 3.5V | Reverse current below breakdown |

**Reading the impedance spec:**
- Z_ZT = 7.0Ω means that for a 1mA change in Zener current, the voltage changes by 7mV
- Lower impedance = better voltage regulation
- Impedance is lower at higher currents (another reason to keep the Zener well into breakdown)

## Checkpoint Questions

1. **A Zener is rated at 5.1V. You apply 3V reverse bias. Does it conduct? What happens?**

2. **Design a Zener regulator: Vin = 15V, Vout = 5.1V, load draws 10mA. Choose Rs so the Zener carries at least 10mA. Calculate the power dissipated in the Zener and the resistor.**

3. **Why is the worst-case power dissipation in a Zener regulator at NO load rather than maximum load?**

4. **A 3.3V Zener has a negative temperature coefficient. A 12V Zener has a positive temperature coefficient. Explain why in terms of breakdown mechanism.**

5. **Your Zener regulator outputs 5.1V at no load but drops to 4.8V at maximum load. What can you do to improve the load regulation?** (Hint: consider the series resistor value and Zener impedance)

6. **Why are Zener regulators not used in high-current (>100mA) applications? What would you use instead?**

7. **The 1N4733A has Z_ZT = 7Ω. If the Zener current changes by 5mA, how much does V_out change?**

---

*Next notebook: [03-reading-a-diode-datasheet.ipynb](03-reading-a-diode-datasheet.ipynb) — Deep dive into reading the 1N4148 datasheet*